In [1]:
from __future__ import print_function

### THIS IS ONLY FOR HYPERPARAMETER TUNING
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

##
## BASELINE
##
from keras.layers import Input, Dense, SimpleRNN, Bidirectional, Dropout, Activation
from keras.callbacks import ReduceLROnPlateau, CSVLogger
from keras.optimizers import Adagrad, adam, sgd, rmsprop
from keras.models import Sequential

import numpy as np


Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
import pickle

def data():
    pkl_file = open('test_X.pkl', 'rb')
    X_test = pickle.load(pkl_file)
    pkl_file.close()
    print ("test_X loaded")

    pkl_file = open('test_y.pkl', 'rb')
    Y_test = pickle.load(pkl_file)
    pkl_file.close()
    print ("test_y loaded")

    pkl_file = open('train_X.pkl', 'rb')
    X_train = pickle.load(pkl_file)
    pkl_file.close()
    print ("train_X loaded")

    pkl_file = open('train_y.pkl', 'rb')
    Y_train = pickle.load(pkl_file)
    pkl_file.close()
    print ("train_y loaded")

    return X_train, Y_train, X_test, Y_test

In [3]:
y_weights = np.load('y_weights.npy').item()
print (y_weights)

{0: 0.4189557794750016, 1: 3.6705746738641474, 2: 2.1043681495809157, 3: 1.1554357830642878}


In [4]:
def model(X_train, Y_train, X_test, Y_test):
    
    # assemble & compile model
    model = Sequential()
    model.add(SimpleRNN(input_shape=(50,68,),units={{choice([25, 50, 100, 150, 200])}},activation={{choice(['relu', 'sigmoid', 'tanh'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', 
                  optimizer={{choice(['rmsprop', 'adam', 'sgd', 'adagrad'])}}, 
                  metrics=['categorical_accuracy'])
    # train
    model.fit(X_train, Y_train,
              batch_size={{choice([50, 100, 500, 1000])}},
              epochs=3,
              shuffle={{choice([True, False])}},
              class_weight = {0: 0.4189557794750016, 1: 3.6705746738641474, 2: 2.1043681495809157, 3: 1.1554357830642878},
              validation_data=(X_test, Y_test),
              verbose=1)
    score, acc = model.evaluate(X_test, Y_test, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

X_train, Y_train, X_test, Y_test = data()
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      notebook_name='optimize')

test_X loaded
test_y loaded
train_X loaded
train_y loaded
>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from keras.layers import Input, Dense, SimpleRNN, Bidirectional, Dropout, Activation
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau, CSVLogger
except:
    pass

try:
    from keras.optimizers import Adagrad, adam, sgd, rmsprop
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import pickle
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'units': hp.choice('units', [25, 50, 100, 150, 200]),
        'activation': hp.choice('activation', ['relu', 'sigmoid', 'tanh']),
        'Dropout': hp.uniform('Dropout', 0, 1),
 

KeyboardInterrupt: 

In [5]:
print(best_run)

NameError: name 'best_run' is not defined

In [ ]:
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))